In [1]:
import numpy as np
import math
import random

In [2]:
class NeuralNetwork():
    def __init__(self, dims):
        self.num_layers = len(dims)
        self.layers = []
        self.bias_layers = []
        for l in range(self.num_layers-1):
            self.layers += [np.zeros((dims[l], dims[l+1]))]
            self.bias_layers += [np.ones((1, dims[l+1]))*-1]          
        pass
    
    def randomize_weights(self, min=-1, max=1):
        for l in range(len(self.layers)):
            for r in range(len(self.layers[l])):
                for c in range(len(self.layers[l][r])):
                    self.layers[l][r,c] = np.random.uniform(min, max)
            for b in range(len(self.bias_layers[l][0])):
                self.bias_layers[l][0][b] = np.random.uniform(min, max)
        pass
    
    def print_layers(self):
        for l,b  in zip(self.layers, self.bias_layers):
            print("layer")
            print(l)
            print("bias")
            print(b)
        print()

In [3]:
def sigmoid(x,k=1):
    return np.array([[1/(1+math.exp(-k*a)) for a in x.ravel()]])

def sigmoid_deriv(x, k=1):
    return np.array([((k*((1+math.e**(-k*a))**(-2)))*(math.e**(-k*a))) for a in x])

def step(x):
    return 1 if x>0 else 0

def ws(x):
    return np.array(x)*0.1

def ws_deriv(x):
    return 0.1

In [4]:
def forward_propagate(NN, x, A):
    x = np.array([x])
    for l in range(len(NN.layers)):
        d = x.dot(NN.layers[l]) + NN.bias_layers[l]
        x = A(d)
    return x
    pass

In [5]:
def error(NN, ts, A):
    e = 0
    for x,y in ts: 
        e += (y - forward_propagate(NN, x, A)[0]).sum()
    return e

def out(NN, ts, A):
    for x,y in ts: 
        print(str(x) + ": " + str(forward_propagate(NN, x, A)))
    print()

In [6]:
MAX_EPOCHS = 600

def backpropagate(ts, dims, A=sigmoid, A_deriv=sigmoid_deriv, lam=10):
    NN = NeuralNetwork(dims)
    NN.randomize_weights()
#     NN.layers = [np.array([[2, 3], [-2, -4], [1, -1]]), np.array([[1, 2], [-2, -1]])]
#     NN.bias_layers = [np.array([[0, 0]]), np.array([[0, 0]])]
    w = [NN.layers[0], NN.layers[1], 0]
    b = [0, NN.bias_layers[0], NN.bias_layers[1]]
    n = NN.num_layers
    a = [0, 0, 0] 
    delta = [0, 0, 0]
    dot = [0, 0, 0]
    
    for e in range(MAX_EPOCHS):
        for x,y in ts:
            a[0] = np.array([x[:]])
            dot[0] = np.array([x[:]])
            for l in range(1, n):
                dot[l] = a[l-1].dot(w[l-1]) + b[l]
                a[l] = A(dot[l])
            delta[n-1] = A_deriv(dot[n-1])*(y-a[n-1])
            for l in range(n-2, 0, -1):
                delta[l] = A_deriv(dot[l])*delta[l+1].dot(np.transpose(w[l]))
            for l in range(n-1):
                w[l] = w[l] + lam*np.transpose(a[l])*delta[l+1]
                b[l+1] = b[l+1] + lam*delta[l+1]
            NN.layers = w[:n-1]
            NN.bias_layers = b[1:]
        if error(NN, ts, A) == 0:
            print("yay")
            return NN
    return NN
    pass

# nn = backpropagate([(np.array([10, 20, 30]), np.array([2, 1]))], dims=[3, 2, 2], A=ws, A_deriv=ws_deriv, lam=100)
# nn.print_layers()

In [7]:
#xor
def padbin(x, n):
    b = str(bin(x))[2:]
    z = ['0' for i in range(n-len(b))]
    return ''.join(z) + b

def make_ts(numbits, f):
    ts = []
    f = [int(i) for i in padbin(f, 2**numbits)]
    for x in range(2**numbits):
        val = np.array([int(i) for i in padbin(x, numbits)])
        ts += [(val, f[x])]
    return ts

In [8]:
t = make_ts(2, 6)
nn = backpropagate(t, [2,2,1])
nn.print_layers()
out(nn, t, A=sigmoid)

layer
[[-5.29899501 -6.85029377]
 [-5.32217803 -7.1160473 ]]
bias
[[7.89079721 2.82322845]]
layer
[[ 9.47303544]
 [-9.72034597]]
bias
[[-4.50457192]]

[0 0]: [[0.01462498]]
[0 1]: [[0.98467867]]
[1 0]: [[0.98429972]]
[1 1]: [[0.01936146]]

